<a href="https://colab.research.google.com/github/mateopolancecofficial/NLP/blob/main/WineReviews/SequenceModelText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import os


In [3]:
pip install -q -U tensorflow-text

     |████████████████████████████████| 3.4MB 6.6MB/s 


In [4]:
pip install -q -U tf-models-official

     |████████████████████████████████| 1.1MB 5.2MB/s 
     |████████████████████████████████| 51kB 3.9MB/s 
     |████████████████████████████████| 645kB 28.6MB/s 
     |████████████████████████████████| 1.2MB 41.8MB/s 
     |████████████████████████████████| 37.6MB 1.7MB/s 
     |████████████████████████████████| 358kB 46.3MB/s 
     |████████████████████████████████| 102kB 8.3MB/s 
     |████████████████████████████████| 706kB 40.3MB/s 
     |████████████████████████████████| 174kB 32.2MB/s 


In [6]:
pip install -U tfds-nightly

Requirement already up-to-date: tfds-nightly in /usr/local/lib/python3.7/dist-packages (4.2.0.dev202104240107)


In [7]:
!git clone -l -s https://github.com/mateopolancecofficial/NLP.git

Cloning into 'NLP'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 33 (delta 8), reused 21 (delta 5), pack-reused 0
Unpacking objects: 100% (33/33), done.


In [22]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text  # A dependency of the preprocessing model
import tensorflow_addons as tfa
from official.nlp import optimization
import numpy as np

tf.get_logger().setLevel('ERROR')

In [23]:
os.environ["TFHUB_MODEL_LOAD_FORMAT"]="UNCOMPRESSED"

In [24]:
import os

if os.environ['COLAB_TPU_ADDR']:
  cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
  tf.config.experimental_connect_to_cluster(cluster_resolver)
  tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
  strategy = tf.distribute.TPUStrategy(cluster_resolver)
  print('Using TPU')
elif tf.test.is_gpu_available():
  strategy = tf.distribute.MirroredStrategy()
  print('Using GPU')
else:
  raise ValueError('Running on CPU is not recommended.')

Using TPU


In [25]:
df = pd.read_csv("/content/NLP/WineReviews/data/winemag-data-130k-v2.csv", index_col=0)
df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [26]:
df = df[['description', 'points']]
df.head()

,description,points
0,"Aromas include tropical fruit, broom, brimston...",87
1,"This is ripe and fruity, a wine that is smooth...",87
2,"Tart and snappy, the flavors of lime flesh and...",87
3,"Pineapple rind, lemon pith and orange blossom ...",87
4,"Much like the regular bottling from 2012, this...",87


In [27]:
df = df.sample(frac = 1)
df.head()

,description,points
301,"A good, clean, tart and lightly spicy all-stai...",87
109461,"This is a beautiful wine, full of ripe yellow ...",95
49873,There's a hardness to this Cabernet that comes...,88
20073,"This is roasted, savory and spicy up front, wi...",87
1812,"A blend of Arinto and Tamarez, this wine is li...",85


In [30]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt



In [31]:
df.description = df.description.str.replace('\d+', '')

In [32]:
from sklearn.model_selection import train_test_split

text_train, text_val, y_train, y_val = train_test_split(df.description, df.points,
                                                  test_size=0.2,train_size=0.8)

In [33]:
model = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)


In [34]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_2 (KerasLayer)   (None, 50)                48190600  
_________________________________________________________________
dense_2 (Dense)              (None, 16)                816       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [35]:
loss = 'mean_squared_error'
metric = 'mean_squared_error'
model.compile(optimizer='adam',
              loss=loss,
              metrics=[metric])

In [ ]:
history = model.fit(text_train,
                    y_train,
                    epochs=40,
                    batch_size=1024,
                    validation_data=(text_val, y_val),
                    verbose=1)

Epoch 1/40
102/102 [==============================] - 18s 174ms/step - loss: 7350.4960 - mean_squared_error: 7350.4960 - val_loss: 3810.9177 - val_mean_squared_error: 3810.9177
Epoch 2/40
102/102 [==============================] - 18s 176ms/step - loss: 2110.5372 - mean_squared_error: 2110.5372 - val_loss: 100.2749 - val_mean_squared_error: 100.2749
Epoch 3/40
102/102 [==============================] - 18s 180ms/step - loss: 96.3244 - mean_squared_error: 96.3244 - val_loss: 89.1597 - val_mean_squared_error: 89.1597
Epoch 4/40
102/102 [==============================] - 18s 179ms/step - loss: 85.7847 - mean_squared_error: 85.7847 - val_loss: 80.9745 - val_mean_squared_error: 80.9745
Epoch 5/40
102/102 [==============================] - 18s 179ms/step - loss: 77.0409 - mean_squared_error: 77.0409 - val_loss: 74.7435 - val_mean_squared_error: 74.7435
Epoch 6/40
102/102 [==============================] - 19s 181ms/step - loss: 69.8673 - mean_squared_error: 69.8673 - val_loss: 70.0609 - val_